## Extraire le texte du pdf

In [62]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()  # Extraire le texte de chaque page
    return text


In [63]:
texte = extract_text_from_pdf("C:/Users/danie/OneDrive/Documents/GitHub/Challenge_web_mining/data/lm/lm_dan.pdf")
print(texte)

Daniella Rakotondratsimba
6 Rue Emile Duport 69009, Lyon
+33 7 86 62 53 12
daniellark11@gmail.com
Métropole de Lyon
Lyon, le 12 Février 2025
Objet : Candidature au poste de Stagiaire Chargé de mission Data – Data BI et Dataviz
Madame, Monsieur,
Actuellement en Master 2 Statistiques et Informatique pour la Science des Données (SISE) à
l’Université Lumière Lyon 2, je suis passionnée par l’analyse et la valorisation des données. Attirée par
les projets à impact direct sur la vie des citoyens, je souhaite rejoindre la Métropole de Lyon en tant
que stagiaire pour participer au développement de tableaux de bord analytiques. Disponible à partir
du 17 Mars 2025 pour une durée de 4 à 6 mois, je suis motivée à contribuer à vos initiatives
d’innovation numérique tout en renforçant mes compétences dans un environnement dynamique et
collaboratif.
Durant ma formation, j’ai acquis une solide expertise en business intelligence et gestion des
données. J’ai travaillé sur des projets impliquant la concep

## Prétraitrement du texte extrait

### nettoyage (minuscule, ponctuation, espace multiple, chiffre)

In [60]:
# import re
# import string

# def clean_text(text):
#     # Convertir le texte en minuscules
#     #cleaned_text = text.lower()
    
#     # Supprimer la ponctuation
#     cleaned_text = text.translate(str.maketrans("", "", string.punctuation))

#     # Supprimer les chiffres
#     cleaned_text = re.sub(r'\d+', '', cleaned_text)  # Cette expression régulière supprime tous les chiffres
    
#     # Supprimer les espaces multiples et les remplacer par un seul espace
#     cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
#     # Supprimer les espaces en début et fin de texte
#     cleaned_text = cleaned_text.strip()
    
#     return cleaned_text



In [59]:
# texte_clean = clean_text(texte)
# print(texte_clean)

### enlever les stopwords

In [58]:
# import nltk
# from nltk.corpus import stopwords

# # Télécharger les stopwords si ce n'est pas encore fait
# nltk.download('stopwords')

In [46]:
# def remove_stopwords(text):
#     stop_words = set(stopwords.words('french'))  # Liste des stopwords en français
#     words = text.split()  # Séparer le texte en mots
#     filtered_words = [word for word in words if word not in stop_words]  # Supprimer les stopwords
#     return ' '.join(filtered_words)

In [47]:


# texte_clean = remove_stopwords(texte_clean)
# print(texte_clean)

## Extraction d'informations

### extraction lieux

In [61]:
# import spacy

# # Charger le modèle spaCy pour le français
# nlp = spacy.load('fr_core_news_md')

# def extract_places(text):
#     doc = nlp(text)
#     # Afficher toutes les entités extraites
#     # for ent in doc.ents:
#     #     print(f"{ent.text}: {ent.label_}")
#     places = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
#     return places

### extraction de compétence - motivation par LLM

In [146]:
from mistralai import Mistral

import requests
import re

# Clé API Mistral (remplace par ta vraie clé)
MISTRAL_API_KEY = "Yp0Uo7Vx4uSJIlc94dj3MA5ME71KpwIR"

# URL de l'API Mistral
API_URL = "https://api.mistral.ai/v1/chat/completions"

# Fonction pour interroger Mistral et extraire des informations
def query_mistral(prompt):
    headers = {
        "Authorization": f"Bearer {MISTRAL_API_KEY}",
        "Content-Type": "application/json"
    }
    
    data = {
        "model": "mistral-medium",  # Utilise mistral-small ou mistral-large si disponible
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7  # Ajuste selon besoin
    }
    
    response = requests.post(API_URL, headers=headers, json=data)
    
    
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        print("Erreur API :", response.text)
        return None

# Fonction pour nettoyer la réponse de l'API et la structurer en liste
def clean_response(response):
    """Nettoie la réponse en supprimant les caractères inutiles et formatant en liste."""
    if not response:
        return []

    # Séparer les éléments par virgule ou retour à la ligne
    items = re.split(r',|\n', response)

    # Nettoyer les espaces et caractères spéciaux
    cleaned_items = [item.strip() for item in items if item.strip()]

    # Supprimer les doublons
    return list(set(cleaned_items))

# Fonction pour extraire les compétences
def extract_skills(text):
    prompt = f"""
    Voici une lettre de motivation :
    "{text}"

    Extrais uniquement les compétences techniques et non techniques sous forme de liste de mots-clés sans phrases ni explication.
    """
    return query_mistral(prompt)

# Fonction pour extraire les motivations
def extract_motivations(text):
    prompt = f"""
    Voici une lettre de motivation :
    "{text}"

    Extrais uniquement les motivations du candidat sous forme de liste de mots-clés sans phrases ni explication.
    """
    return query_mistral(prompt)

# Fonction pour extraire le lieu de disponibilité
def extract_location(text):
    prompt = f"""
    Voici une lettre de motivation :
    "{text}"

    Identifie et retourne uniquement le nom du lieu où le candidat indique être disponible pour travailler, sans phrase ni explication.
    """
    return query_mistral(prompt)




In [145]:
skills = extract_skills(texte)
print("✅ Compétences :", skills)

TypeError: Object of type Response is not JSON serializable

In [119]:
motivations = extract_motivations(texte)
print("✅ Motivations :", motivations)

✅ Motivations : 1. Passionnée par l'analyse et la valorisation des données
2. Projets à impact direct sur la vie des citoyens
3. Stagiaire pour participer au développement de tableaux de bord analytiques
4. Contribuer à vos initiatives d’innovation numérique
5. Renforcer ses compétences dans un environnement dynamique et collaboratif
6. Solide expertise en business intelligence et gestion des données
7. Conception de tableaux de bord interactifs
8. Préparation et traitement de données complexes
9. Mise en œuvre de KPIs adaptés aux besoins métiers
10. Utilisation d’outils tels que Power BI, SQL et des processus ETL
11. Concevoir des visualisations claires et optimisées pour faciliter la prise de décision
12. Excellent sens de l’analyse
13. Rigoureuse, curieuse et à l’aise dans le travail en équipe
14. Aisance relationnelle
15. Collaborer efficacement avec des équipes multidisciplinaires
16. Adapter aux enjeux spécifiques des différents métiers
17. Engagement à comprendre et répondre aux

In [120]:
location = extract_location(texte)
print("📍 Localisation :", location)

📍 Localisation : Lyon

Note: The candidate mentions that they want to join the Métropole de Lyon, which suggests they are available to work in Lyon. However, the exact location within Lyon is not specified. Therefore, I have returned "Lyon" as the place where the candidate is available to work. If you need more specific information, please let me know.


## Insertion dans la bdd

In [139]:
import psycopg2

# Configuration de la connexion PostgreSQL
DB_CONFIG = {
    "dbname": "cv_lm_db",
    "user": "postgres",
    "password": "daniella",
    "host": "localhost",
    "port": 5432
}

def get_db_connection():
    """Ouvre une connexion à PostgreSQL."""
    return psycopg2.connect(**DB_CONFIG)


### enregistrer les données du LM dans la bdd

In [142]:
def insert_competences(competences):
    """Insère des compétences dans la table des compétences."""
    connection = get_db_connection()
    cursor = connection.cursor()

    try:
        # Format des compétences en un tableau
        competences_list = competences.split('\n')  # Supposons que vous sépariez par ligne
        formatted_competences = '{' + ','.join(competences_list) + '}'

        # Insertion du tableau
        cursor.execute("""
        INSERT INTO competences (competences)
        VALUES (%s)
        """, (formatted_competences,))
        
        # Commit les changements
        connection.commit()
        print("Compétences insérées avec succès.")
    except Exception as e:
        print(f"Erreur lors de l'insertion des compétences : {e}")
    finally:
        cursor.close()
        connection.close()


In [143]:
insert_competences(skills)

Erreur lors de l'insertion des compétences : ERREUR:  tableau litéral mal formé : « {Compétences techniques :,,* Business intelligence,* Gestion de données,* Conception de tableaux de bord interactifs,* Préparation de données,* Traitement de données complexes,* Mise en œuvre de KPIs,* Power BI,* SQL,* Processus ETL,* Visualisations de données,* Migration de tableaux de bord,* Formation d'utilisateurs finaux,,Compétences non techniques :,,* Sens de l'analyse,* Rigueur,* Curiosité,* Travail en équipe,* Aisance relationnelle,* Adaptabilité,* Engagement utilisateur} »
LINE 3:         VALUES ('{Compétences techniques :,,* Business intel...
                        ^
DETAIL:  Caractère « , » inattendu.

